<font size="5">On-Chip Edge Learning</font>

This notebook was executed on the development kit from BrainChip. Furthermore, the on-chip edge learning was executed. The code was inspired by the official code from BrainChip (https://doc.brainchipinc.com/examples/edge/plot_1_edge_learning_kws.html).

In [1]:
import os
from os import listdir
from os.path import isdir, join
import pathlib

import akida
from akida import FullyConnected
from akida import evaluate_sparsity
import cnn2snn
from cnn2snn import check_model_compatibility
from cnn2snn import quantize
from cnn2snn import quantize_layer
from cnn2snn import convert

from math import ceil

from timeit import default_timer as timer
import numpy as np

import itertools
import matplotlib.pyplot as plt

<font size="5"> 1. Load the Data Set and Model</font>

In [2]:
# Load the previous trained model. (Source Model: Final_OnChip_Edge)

akida_model = akida.Model('akida_model_edge.fbz')

In [3]:
# Define directories of the data

feature_sets_filename = 'final_stored_files_targets_int_normalized_wedge.npz'
feature_sets_filename_edge = 'final_stored_files_targets_int_normalized_just_edge.npz'

# Load feature sets

feature_sets = np.load(feature_sets_filename)
feature_sets_edge = np.load(feature_sets_filename_edge)
print('Feature Sets: ', feature_sets.files)
print('Feature Sets for Edge: ', feature_sets_edge.files)

Feature Sets:  ['x_train', 'y_train', 'x_val', 'y_val', 'x_test', 'y_test']
Feature Sets for Edge:  ['x_train', 'y_train', 'x_val', 'y_val']


In [4]:
# Assign feature sets of inital data set

x_train = feature_sets['x_train']
y_train = feature_sets['y_train']
x_val = feature_sets['x_val']
y_val = feature_sets['y_val']
x_test = feature_sets['x_test']
y_test = feature_sets['y_test']

# Assign feature sets of edge data set

x_train_edge = feature_sets_edge['x_train']
y_train_edge = feature_sets_edge['y_train']
x_val_edge = feature_sets_edge['x_val']
y_val_edge = feature_sets_edge['y_val']

In [5]:
# CNN for conversion expects (batch, height, width, channels)
# The channels can either be 1 for gray-scaled images or 3 for RGB-images

x_train = x_train.reshape(x_train.shape[0], 
                          x_train.shape[1], 
                          x_train.shape[2], 
                          1)
x_val = x_val.reshape(x_val.shape[0], 
                      x_val.shape[1], 
                      x_val.shape[2], 
                      1)
x_test = x_test.reshape(x_test.shape[0], 
                        x_test.shape[1], 
                        x_test.shape[2], 
                        1)

x_train_edge = x_train_edge.reshape(x_train_edge.shape[0], 
                          x_train_edge.shape[1], 
                          x_train_edge.shape[2], 
                          1)
x_val_edge = x_val_edge.reshape(x_val_edge.shape[0], 
                      x_val_edge.shape[1], 
                      x_val_edge.shape[2], 
                      1)



print('Add dimension to all data: Order Train, Val, Test, Train_edge, Val_edge: ')
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)
print(x_train_edge.shape)
print(x_val_edge.shape)

Add dimension to all data: Order Train, Val, Test, Train_edge, Val_edge: 
(34972, 40, 101, 1)
(4371, 40, 101, 1)
(4371, 40, 101, 1)
(120, 40, 101, 1)
(19, 40, 101, 1)


<font size="5"> 2. Edge Learning with previously defined Hyperparameters</font>

The hyperparameters were defined in the notebook Final_OnChip_Edge.

In [6]:
devices = akida.devices()
print('Akida NP available :', devices[0])
device = devices[0]

Akida NP available : <akida.core.HardwareDevice object at 0xffff430ca630>


In [7]:
akida_model.map(device)

In [8]:
# Add 3 classes

akida_model.add_classes(3)

In [9]:
# Train the Akida on the new keywords
# Code Source: https://doc.brainchipinc.com/examples/edge/plot_1_edge_learning_kws.html

print("\nEdge learning with 3 new classes ...")
start = timer()
akida_model.fit(x_train_edge, y_train_edge.astype(np.int32))
end = timer()
print(f"Elapsed time for Akida edge learning: {end-start:.2f} s")


Edge learning with 3 new classes ...
Elapsed time for Akida edge learning: 0.85 s


In [11]:
# Define the number of keywords used in the edge data set

num_labels_edge = 15

In [12]:
# Print the performance on the different data sets

preds_val_ak_old_train = akida_model.predict(x_train, num_classes=num_labels_edge)
acc_val_ak = np.sum(preds_val_ak_old_train == y_train) / y_train.shape[0]
print(f"Akida train set accuracy on old data set: {100 * acc_val_ak:.2f} %")
preds_val_ak_old_val = akida_model.predict(x_val, num_classes=num_labels_edge)
acc_val_ak = np.sum(preds_val_ak_old_val == y_val) / y_val.shape[0]
print(f"Akida validation set accuracy on old data set: {100 * acc_val_ak:.2f} %")
preds_val_ak_old_test = akida_model.predict(x_test, num_classes=num_labels_edge)
acc_val_ak = np.sum(preds_val_ak_old_test == y_test) / y_test.shape[0]
print(f"Akida test set accuracy on old data set: {100 * acc_val_ak:.2f} %")

Akida train set accuracy on old data set: 98.97 %
Akida validation set accuracy on old data set: 94.01 %
Akida test set accuracy on old data set: 93.46 %


In [16]:
# Print the performance on the edge data set. 

preds_val_ak_new_train = akida_model.predict(x_train_edge, num_classes=num_labels_edge)
acc_val_ak = np.sum(preds_val_ak_new_train == y_train_edge) / y_train_edge.shape[0]
print(f"Akida training set accuracy on new data set: {100 * acc_val_ak:.2f} %")
preds_val_ak_new_val = akida_model.predict(x_val_edge, num_classes=num_labels_edge)
acc_val_ak = np.sum(preds_val_ak_new_val == y_val_edge) / y_val_edge.shape[0]
print(f"Akida validation set accuracy on new data set: {100 * acc_val_ak:.2f} %")

Akida training set accuracy on new data set: 94.17 %
Akida validation set accuracy on new data set: 89.47 %
